In [1]:
import pandas as pd
from shapely.geometry import MultiPoint

In [193]:
# read the csv file containing borderline longitudes and latitudes for all Indian districts
df = pd.read_csv(r"C:\Users\samsu\Desktop\delete\Ind_adm2_Points (1).csv")

In [190]:
df.head()

,Ind_adm2_ID,sub_polygon_id,point_order,Latitude,Longitude,Country,State,District
0,0,1,1,10.8976,92.5158,India,Andaman and Nicobar,Andaman Islands
1,0,1,2,10.8975,92.5161,India,Andaman and Nicobar,Andaman Islands
2,0,1,3,10.8975,92.5164,India,Andaman and Nicobar,Andaman Islands
3,0,1,4,10.8972,92.5167,India,Andaman and Nicobar,Andaman Islands
4,0,1,5,10.8972,92.5203,India,Andaman and Nicobar,Andaman Islands


In [122]:
district = ""
geoList = []
result_df = pd.DataFrame(data=None,columns=['State','District','Latitude','Longitude'])

for index, row in df.iterrows():
    # check if this is anew district value
    if district and (district!=df.iloc[index]['District']):
        # calculate centroid for previous district
        points = MultiPoint(geoList)
        # save the state, district, long-lat and centroid to new dataframe
        result_df = result_df.append({'State':df['State'].iloc[index-1],'District':df['District'].iloc[index-1],'Latitude':points.centroid.x,'Longitude':points.centroid.y}, ignore_index=True)
        # clear old geoList (APPEND NEW LONG-LAT ALSO)
        del geoList[:]
    # save this new district's name
    district = df.iloc[index]['District']
    # add this long lat info to later calculate centroid
    geoList.append((df.iloc[index]['Latitude'],df.iloc[index]['Longitude']))

# add last district's centroid
if geoList:
    points = MultiPoint(geoList)
    result_df = result_df.append({'State':df['State'].iloc[-1],'District':df['District'].iloc[-1],'Latitude':points.centroid.x,'Longitude':points.centroid.y}, ignore_index=True)
    del geoList[:]

In [124]:
result_df.head()

,State,District,Latitude,Longitude
0,Andaman and Nicobar,Andaman Islands,12.382571,92.822911
1,Andaman and Nicobar,Nicobar Islands,7.835291,93.511601
2,Andhra Pradesh,Adilabad,19.284514,78.813212
3,Andhra Pradesh,Anantapur,14.312066,77.460158
4,Andhra Pradesh,Chittoor,13.331093,78.927639


In [129]:
result_df.to_csv("centroids.csv",index=False)

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

In [7]:
res=requests.get('http://www.citypopulation.de/India.html')

In [9]:
soup= bs(res.text,'lxml')

In [50]:
allLinks = soup.select('ul li a')

In [52]:
allLinks = allLinks[int(len(allLinks)/2):]

In [60]:
state = []
links = []
for item in allLinks:
    state.append(item.text)
    links.append("http://www.citypopulation.de/"+item.attrs['href'])

In [70]:
population_df = pd.DataFrame(data=None,columns=['State','District','Population in 2001','Population in 2011'])
for idx, link in enumerate(links):
    temp_df = pd.read_html(link)[0]
    for index, row in temp_df.iterrows():
        if temp_df['Status'].iloc[index] == 'District':
            population_df = population_df.append({'State':state[idx],'District':temp_df['Name'].iloc[index],'Population in 2001':temp_df['PopulationCensus2001-03-01'].iloc[index],'Population in 2011':temp_df['PopulationCensus2011-03-01'].iloc[index]}, ignore_index=True)

In [73]:
population_df.to_csv("population.csv",index=False)

In [192]:
population_df.head()

,State,District,Population in 2001,Population in 2011
0,Andaman & Nicobar Islands,Nicobar,42068,36842
1,Andaman & Nicobar Islands,North & Middle Andaman,105613,105597
2,Andaman & Nicobar Islands,South Andaman,208471,238142
3,Andhra Pradesh,Anantapur,3640478,4081148
4,Andhra Pradesh,Chittoor,3745875,4174064


In [84]:
merged_df = pd.merge(df,population_df,on=['District','State'],how='left')

In [86]:
merged_df.to_csv('merged_withNA.csv',index=False)

In [87]:
merged_df.dropna(axis=0,how='any').to_csv('merged_withoutNA.csv',index=False)

In [168]:
input_df = pd.read_csv(r"C:\Users\samsu\Desktop\delete\merged_withoutNA.csv")

In [169]:
input_df.head()

,State,District,Latitude,Longitude,Population in 2001,Population in 2011
0,Andaman and Nicobar,Andaman Islands,12.382571,92.822911,314084,343739
1,Andaman and Nicobar,Nicobar Islands,7.835291,93.511601,42068,36842
2,Andhra Pradesh,Anantapur,14.312066,77.460158,3640478,4081148
3,Andhra Pradesh,Chittoor,13.331093,78.927639,3745875,4174064
4,Andhra Pradesh,Cuddapah,14.490289,78.696139,2601797,2882469


In [176]:
state = ""
weights = []
x_numerator = []
y_numerator = []
results = pd.DataFrame(data=None,columns=['State','Longitude','Latitude'])

for index,x in input_df.iterrows():
    if state and (state!=input_df.iloc[index]['State']):
        #calculate weighted mean for the last state
        longitude = sum(x_numerator)/sum(weights)
        latitude = sum(y_numerator)/sum(weights)
        #empty lists
        del x_numerator[:]
        del y_numerator[:]
        del weights[:]
        #store to results dataframe
        results = results.append({'State':state,'Longitude':longitude,'Latitude':latitude}, ignore_index=True)
    state = input_df.iloc[index]['State']
    weights.append(float(input_df.iloc[index]['Population in 2001']))
    x_numerator.append(float(input_df.iloc[index]['Longitude'])*float(input_df.iloc[index]['Population in 2001']))
    y_numerator.append(float(input_df.iloc[index]['Latitude'])*float(input_df.iloc[index]['Population in 2001']))
if weights:
    longitude = sum(x_numerator)/sum(weights)
    latitude = sum(y_numerator)/sum(weights)
    del x_numerator[:]
    del y_numerator[:]
    del weights[:]
    results = results.append({'State':state,'Longitude':longitude,'Latitude':latitude}, ignore_index=True)

In [177]:
results.to_csv(r'C:\Users\samsu\Desktop\delete\results_2001.csv',index=False)

In [178]:
state = ""
weights = []
x_numerator = []
y_numerator = []
results = pd.DataFrame(data=None,columns=['State','Longitude','Latitude'])

for index,x in input_df.iterrows():
    if state and (state!=input_df.iloc[index]['State']):
        #calculate weighted mean for the last state
        longitude = sum(x_numerator)/sum(weights)
        latitude = sum(y_numerator)/sum(weights)
        #empty lists
        del x_numerator[:]
        del y_numerator[:]
        del weights[:]
        #store to results dataframe
        results = results.append({'State':state,'Longitude':longitude,'Latitude':latitude}, ignore_index=True)
    state = input_df.iloc[index]['State']
    weights.append(float(input_df.iloc[index]['Population in 2011']))
    x_numerator.append(float(input_df.iloc[index]['Longitude'])*float(input_df.iloc[index]['Population in 2011']))
    y_numerator.append(float(input_df.iloc[index]['Latitude'])*float(input_df.iloc[index]['Population in 2011']))
if weights:
    longitude = sum(x_numerator)/sum(weights)
    latitude = sum(y_numerator)/sum(weights)
    del x_numerator[:]
    del y_numerator[:]
    del weights[:]
    results = results.append({'State':state,'Longitude':longitude,'Latitude':latitude}, ignore_index=True)

In [179]:
results.to_csv(r'C:\Users\samsu\Desktop\delete\results_2011.csv',index=False)